# VERSPM Model Interface

In [ ]:
import emat
import os

This notebook is meant to illustrate the use of TMIP-EMAT's
with VisionEval's RSPM Model.  It provides an illustration of how to use 
TMIP-EMAT and the interface to run the model.

In this example notebook, we will activate some logging features.  The 
same logging utility is written directly into the EMAT and the
`emat_verspm.py` module. This will give us a view of what's happening
inside the code as it runs.

In [ ]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.INFO)

## Connecting to the Model

The interface for this model is located in the `emat_verspm.py`
module, which we will import into this notebook. 

In [ ]:
import emat_verspm

Let's initialize a database file to store results.

In [ ]:
database_path = os.path.expanduser("~/EMAT-VE/ve2-rspm-2020-10-17.db")
initialize = not os.path.exists(database_path)
db = emat.SQLiteDB(database_path, initialize=initialize)

Within this module, you will find a definition for the 
`VERSPModel` class.  We initialize an instance of the model interface object.

In [ ]:
fx = emat_verspm.VERSPModel(db=db)

## Single Run Operation for Development and Debugging

Before we take on the task of running this model in exploratory mode, we'll
want to make sure that our interface code is working correctly. To check each
of the components of the interface (setup, run, post-process, load-measures,
and archive), we can run each individually in sequence, and inspect the results
to make sure they are correct.

### setup

This method is the place where the core model *set up* takes place,
including creating or modifying files as necessary to prepare
for a core model run.  When running experiments, this method
is called once for each core model experiment, where each experiment
is defined by a set of particular values for both the exogenous
uncertainties and the policy levers.  These values are passed to
the experiment only here, and not in the `run` method itself.
This facilitates debugging, as the `setup` method can be used 
without the `run` method, as we do here. This allows us to manually
inspect the prepared files and ensure they are correct before
actually running a potentially expensive model.

Each input exogenous uncertainty or policy lever can potentially
be used to manipulate multiple different aspects of the underlying
core model.  For example, a policy lever that includes a number of
discrete future network "build" options might trigger the replacement
of multiple related network definition files.  Or, a single uncertainty
relating to the cost of fuel might scale both a parameter linked to
the modeled per-mile cost of operating an automobile and the
modeled total cost of fuel used by transit services.

In our RSPM module's `setup`, parameters that are omitted are set at their
deafult values, but we can give a subset of parameters with non-default values
if we like.

In [ ]:
params = {
    'ValueOfTime': 13,
    'Income': 46300,
    'Transit': 1.34,
    'ElectricCost': 0.14,
    'FuelCost': 4.25,
} 

fx.setup(params)

### run

The `run` method is the place where the core model run takes place.
Note that this method takes no arguments; all the input
exogenous uncertainties and policy levers are delivered to the
core model in the `setup` method, which will be executed prior
to calling this method. This facilitates debugging, as the `setup`
method can be used without the `run` method as we did above, allowing
us to manually inspect the prepared files and ensure they
are correct before actually running a potentially expensive model.

In [ ]:
fx.run()

The `VERSPModel` class includes a custom `last_run_logs` method,
which displays both the "stdout" and "stderr" logs generated by the 
model executable during the most recent call to the `run` method.
We can use this method for debugging purposes, to identify why the 
core model crashes (if it does crash).  In this first test it did not
crash, and the logs look good.

In [ ]:
fx.last_run_logs()

If we look in the model's output directory, we see all the output files we need.

In [ ]:
from emat.util.show_dir import show_dir
show_dir(os.path.join(fx.master_directory.name, 'VERSPM', 'output'))

### post-process

There is a `post_process` step that is separate from the `run` step.

For VERSPM, the post-processing replicates the calculations needed to
create some of the same summary performance measures as the _R_ version of
VisionEval does when run with scenarios.

In [ ]:
fx.post_process()

### load-measures

The `load_measures` method is the place to actually reach into
files in the RSPM's run results and extract particular performance
measures, returning a dictionary of key-value pairs for the 
various performance measures.  

In [ ]:
fx.load_measures()

### archive

The `archive` method copies the relevant model output files to an archive location for 
longer term storage.  The particular archive location is based on the experiment id
for a particular experiment plus a unique run id, and can be customized if desired by overloading the 
`get_experiment_archive_path` method.  This customization is not done in this demo,
so the default location is used.

Actually running the `archive` method should copy any relevant output files
from the `model_path` of the current active model into a subdirectory of `archive_path`.

In [ ]:
fx.archive(params)

It is permissible, but not required, to simply copy the entire contents of the 
former to the latter, as is done in this example. However, if the current active model
directory has a lot of boilerplate files that don't change with the inputs, or
if it becomes full of intermediate or temporary files that definitely will never
be used to compute performance measures, it can be advisable to selectively copy
only relevant files. In that case, those files and whatever related sub-directory
tree structure exists in the current active model should be replicated within the
experiments archive directory.

## Running a Single Reference Experiment

Once we have finished the development of our interface, we don't need to worry about 
running these individual parts. Instead, we can simply trigger one or more full 
core model RSPM runs from a single command.

For example, we can run a reference experiment with all default values to establish 
a baseline set of results.

In [ ]:
fx.run_reference_experiment()

## Single Thread Operation for Running Multiple Experiments

For this demo, we'll create a design of experiments with only 3 experiments.
The `design_experiments` method of the `VERSPModel` object is not defined
in the custom code written for this model, but rather is a generic
function provide by the TMIP-EMAT main library.
Real applications will typically use a larger number of experiments, but this small number
is sufficient to demonstrate the operation of the tools.

In [ ]:
design1 = fx.design_experiments(n_samples=3)
design1

The `run_experiments` command will automatically run the model once for each experiment in the named design.

In [ ]:
fx.run_experiments(design1)

## Multiprocessing for Running Multiple Experiments

The examples above are all single-process demonstrations of using TMIP-EMAT to run core model
VERSPM experiments. The RSPM model itself is single-threaded, but you can run multiple independent instances of
the model side-by-side on the same machine, so you can benefit from a multiprocessing 
approach.  This can be accomplished by splitting a design of experiments over several
processes that you start manually, or by using an automatic multiprocessing library such as 
`dask.distributed`.

In [ ]:
design2 = fx.design_experiments(random_seed=3, sampler='ulhs')
design2

The module is set up to facilitate distributed multiprocessing. During the `setup`
step, the code detects if it is being run in a distributed "worker" environment instead of
in a normal Python environment.  If the "worker" environment is detected, then a copy
of the entire VERSPM model is made into the worker's local workspace, and the model
is run there instead of in the master workspace.  This allows each worker to edit the files
independently and simultaneously, without disturbing other parallel workers.

To run the model with parallel subprocesses, we can use the asynchronous evaluator
built into TMIP-EMAT. This model running engine will start background processes and 
run the model without locking up this Jupyter Notebook, so we can interactively inspect
the progress of the model runs while they are running.

In [ ]:
work = fx.async_experiments(design=design2, db=db, batch_size=1)

In [ ]:
work.progress()

In [ ]:
work.progress()

In [ ]:
work.progress()

In [ ]:
fx.db.read_design_names(None)

In [ ]:
fx.db.read_experiment_all(None, runs='all', with_run_ids=True)